In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e7/sample_submission.csv
/kaggle/input/playground-series-s5e7/train.csv
/kaggle/input/playground-series-s5e7/test.csv


In [2]:
df = pd.read_csv('/kaggle/input/playground-series-s5e7/train.csv')
df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,0,0.0,No,6.0,4.0,No,15.0,5.0,Extrovert
1,1,1.0,No,7.0,3.0,No,10.0,8.0,Extrovert
2,2,6.0,Yes,1.0,0.0,NaN,3.0,0.0,Introvert
3,3,3.0,No,7.0,3.0,No,11.0,5.0,Extrovert
4,4,1.0,No,4.0,4.0,No,13.0,NaN,Extrovert
...,...,...,...,...,...,...,...,...,...
18519,18519,3.0,No,7.0,3.0,No,9.0,7.0,Extrovert
18520,18520,1.0,NaN,6.0,7.0,No,6.0,5.0,Extrovert
18521,18521,7.0,Yes,1.0,1.0,Yes,1.0,NaN,Introvert
18522,18522,NaN,Yes,1.0,0.0,Yes,5.0,2.0,Introvert


In [3]:
df.columns

Index(['id', 'Time_spent_Alone', 'Stage_fear', 'Social_event_attendance',
       'Going_outside', 'Drained_after_socializing', 'Friends_circle_size',
       'Post_frequency', 'Personality'],
      dtype='object')

In [4]:
np.shape(df)

(18524, 9)

In [5]:
df.isnull().sum()

id                              0
Time_spent_Alone             1190
Stage_fear                   1893
Social_event_attendance      1180
Going_outside                1466
Drained_after_socializing    1149
Friends_circle_size          1054
Post_frequency               1264
Personality                     0
dtype: int64

In [6]:
"""Competition: Kaggle Playground Series S5E7 | Date: July 2, 2025 | Purpose: XGBoost with In-Fold Preprocessing and Probability Calibration"""
# Install statements
# To run this code, ensure you have the following libraries installed.
# You can install them using pip:
# !pip install pandas numpy scikit-learn xgboost matplotlib seaborn

import warnings
warnings.filterwarnings('ignore') # Suppress all warnings for cleaner output

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score # Competition metric
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.calibration import CalibratedClassifierCV # Import the new library
import os
import gc

# --- Data Loading ---
print("--- Starting Data Loading ---")
# Assuming DATA_DIR is set correctly for your environment
# For Kaggle notebooks, it's typically '/kaggle/input/playground-series-s5e7/'
DATA_DIR = '/kaggle/input/playground-series-s5e7/'
TRAIN_FILE = os.path.join(DATA_DIR, 'train.csv')
TEST_FILE = os.path.join(DATA_DIR, 'test.csv')

# Initialize dataframes to empty to prevent NameError if loading fails
train_df = pd.DataFrame()
test_df = pd.DataFrame()

try:
    print(f"Loading train data from: {TRAIN_FILE}")
    train_df = pd.read_csv(TRAIN_FILE)
    print(f"Train data loaded. Shape: {train_df.shape}")

    print(f"Loading test data from: {TEST_FILE}")
    test_df = pd.read_csv(TEST_FILE)
    print(f"Test data loaded. Shape: {test_df.shape}")
    
    if 'Personality' in test_df.columns:
        test_df = test_df.drop('Personality', axis=1)
        print("Dropped 'Personality' column from test_df as it is the target variable.")

except Exception as e:
    print(f"Error loading data: {e}")
    # Fallback for local execution if the above path fails
    try:
        print("Attempting to load from local directory './data/'")
        train_df = pd.read_csv('train.csv')
        test_df = pd.read_csv('test.csv')
        print("Loaded data successfully from local directory.")
        if 'Personality' in test_df.columns:
            test_df = test_df.drop('Personality', axis=1)

    except Exception as local_e:
        print(f"Failed to load data from local directory as well: {local_e}")
        print("Please ensure train.csv and test.csv are in the correct path.")
        exit()


if train_df.empty or test_df.empty:
    print("Train or test dataframe is empty. Exiting.")
    exit()

# Store test_id and train_id for OOF and submission files
test_id = test_df['id']
train_id = train_df['id']
train_df = train_df.drop('id', axis=1)
test_df = test_df.drop('id', axis=1)


# --- Add Constant Feature to Synthetic Data Only ---
print("\n" + "="*50)
print("--- Adding Constant Feature to Synthetic Datasets ---")
print("="*50)
CONSTANT_FEATURE_NAME = 'constant_zero_feature'
CONSTANT_FEATURE_VALUE = 0
train_df[CONSTANT_FEATURE_NAME] = CONSTANT_FEATURE_VALUE
test_df[CONSTANT_FEATURE_NAME] = CONSTANT_FEATURE_VALUE
print(f"Added '{CONSTANT_FEATURE_NAME}' with value {CONSTANT_FEATURE_VALUE} to train_df and test_df.")

# --- Prepare Data for XGBoost ---
print("\n--- Preparing Data for Model Training ---")
TARGET = 'Personality'

if TARGET not in train_df.columns:
    raise KeyError(f"The target column '{TARGET}' is not found in train_df.")

y = train_df[TARGET]
X = train_df.drop(columns=[TARGET])
X_test = test_df.copy() # Use a copy to avoid modifying the original test set

le_target = LabelEncoder()
y_encoded = le_target.fit_transform(y)
print(f"Target variable encoded: {le_target.classes_} mapped to {le_target.transform(le_target.classes_)}")

# --- Best Hyperparameters from Previous Optuna Search (Hardcoded) ---
best_lr = 0.006358
best_params = {
    'max_depth': 8, 'subsample': 0.8854, 'colsample_bytree': 0.6000,
    'reg_lambda': 0.8295, 'reg_alpha': 5.5149, 'gamma': 0.0395, 'min_child_weight': 2
}
FIXED_N_ESTIMATORS = 1000

final_xgb_params = {
    'objective': 'binary:logistic', 'eval_metric': 'logloss',
    'tree_method': 'gpu_hist', 'predictor': 'gpu_predictor',
    'random_state': 42, 'n_estimators': FIXED_N_ESTIMATORS,
    'learning_rate': best_lr, **best_params,
    'enable_categorical': False
}

# --- Model Training and In-Fold Preprocessing ---
print("\n" + "="*50)
print("--- Starting Final Model Training with Probability Calibration ---")
print("="*50)

N_SPLITS_FINAL_CV = 5
kf_final = StratifiedKFold(n_splits=N_SPLITS_FINAL_CV, shuffle=True, random_state=42)

oof_preds = np.zeros(len(y_encoded))
test_preds_folds = []
fold_accuracies = []
feature_importance_df = None

numerical_cols = X.select_dtypes(include=np.number).columns.tolist()
categorical_cols = X.select_dtypes(include='object').columns.tolist()

for fold, (train_idx, val_idx) in enumerate(kf_final.split(X, y_encoded)):
    print(f"--- Training Fold {fold+1}/{N_SPLITS_FINAL_CV} ---")
    
    X_train, X_val = X.iloc[train_idx].copy(), X.iloc[val_idx].copy()
    y_train, y_val = y_encoded[train_idx], y_encoded[val_idx]
    X_test_fold = X_test.copy()

    imputer = IterativeImputer(max_iter=10, random_state=42, initial_strategy='median')
    X_train[numerical_cols] = imputer.fit_transform(X_train[numerical_cols])
    X_val[numerical_cols] = imputer.transform(X_val[numerical_cols])
    X_test_fold[numerical_cols] = imputer.transform(X_test_fold[numerical_cols])

    for col in categorical_cols:
        X_train[col] = X_train[col].fillna('Missing')
        X_val[col] = X_val[col].fillna('Missing')
        X_test_fold[col] = X_test_fold[col].fillna('Missing')

    X_train = pd.get_dummies(X_train, columns=categorical_cols, drop_first=False)
    X_val = pd.get_dummies(X_val, columns=categorical_cols, drop_first=False)
    X_test_fold = pd.get_dummies(X_test_fold, columns=categorical_cols, drop_first=False)

    train_cols = X_train.columns
    X_val = X_val.reindex(columns=train_cols, fill_value=0)
    X_test_fold = X_test_fold.reindex(columns=train_cols, fill_value=0)
    
    # # Insert after line 186:
    # 5. Model Definition with Calibration
    # Define the base XGBoost model
    base_model = xgb.XGBClassifier(**final_xgb_params)
    
    # Wrap the base model with CalibratedClassifierCV
    # This will fit the base_model and then a calibrator on its predictions.
    # The 'cv' parameter here is for the calibrator's internal cross-validation.
    # Using a new StratifiedKFold for the calibrator is a robust practice.
    calibrator_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=101)
    model_fold = CalibratedClassifierCV(base_model, method='isotonic', cv=calibrator_cv)
    print("Model wrapped with CalibratedClassifierCV (isotonic).")
    
    model_fold.fit(X_train, y_train)
                   
    fold_oof_preds = model_fold.predict(X_val)
    oof_preds[val_idx] = fold_oof_preds
    
    fold_acc = accuracy_score(y_val, fold_oof_preds)
    fold_accuracies.append(fold_acc)
    print(f"Fold {fold+1} Accuracy: {fold_acc:.4f}")

    test_preds_folds.append(model_fold.predict_proba(X_test_fold)[:, 1])
    
    # Feature importance must be extracted from the base model of the first calibrated estimator
    if hasattr(model_fold, 'calibrated_classifiers_'):
        # Access the underlying base model from the first fold of the calibrator
        base_estimator = model_fold.calibrated_classifiers_[0].estimator
        fold_importances = pd.DataFrame({'Feature': train_cols, 'Importance': base_estimator.feature_importances_})
        if feature_importance_df is None:
            feature_importance_df = fold_importances
        else:
            # Use a unique suffix for each fold to avoid column name collisions
            feature_importance_df = feature_importance_df.merge(fold_importances, on='Feature', how='outer', suffixes=('', f'_fold{fold+1}'))

    del X_train, X_val, y_train, y_val, X_test_fold, model_fold, base_model, imputer
    gc.collect()

# --- Post-Training Analysis and Submission File Generation ---
print("\n" + "="*50)
print("--- Finalizing Results ---")
print("="*50)

final_test_preds_proba = np.mean(test_preds_folds, axis=0)
final_test_preds_int = (final_test_preds_proba > 0.5).astype(int)

final_cv_accuracy = accuracy_score(y_encoded, oof_preds)
print(f"\nOverall Final Model CV Accuracy: {final_cv_accuracy:.4f}")

oof_df = pd.DataFrame({'id': train_id, 'oof_preds_class': oof_preds, 'target': y_encoded})
oof_file_path = 'oof_predictions.csv'
oof_df.to_csv(oof_file_path, index=False)
print(f"\nOOF predictions saved to: {oof_file_path}")

final_test_predictions_labels = le_target.inverse_transform(final_test_preds_int)
submission_df = pd.DataFrame({'id': test_id, 'Personality': final_test_predictions_labels})
submission_file_path = 'submission.csv'
submission_df.to_csv(submission_file_path, index=False)
print(f"\nSubmission file created at: {submission_file_path}")
print(f"Sample Submission Head:\n{submission_df.head()}")

# --- Feature Importance Analysis ---
# Fill NaNs that may result from the outer merge before calculating the mean
feature_importance_df = feature_importance_df.fillna(0)
importance_cols = [col for col in feature_importance_df.columns if 'Importance' in col]
feature_importance_df['Mean_Importance'] = feature_importance_df[importance_cols].mean(axis=1)
feature_importance_df = feature_importance_df[['Feature', 'Mean_Importance']].sort_values(by='Mean_Importance', ascending=False)

print("\nFeature Importances (Mean across CV folds):")
print(feature_importance_df.head(20))

print("\n--- Final Model Training and Prediction Complete ---")

--- Starting Data Loading ---
Loading train data from: /kaggle/input/playground-series-s5e7/train.csv
Train data loaded. Shape: (18524, 9)
Loading test data from: /kaggle/input/playground-series-s5e7/test.csv
Test data loaded. Shape: (6175, 8)

--- Adding Constant Feature to Synthetic Datasets ---
Added 'constant_zero_feature' with value 0 to train_df and test_df.

--- Preparing Data for Model Training ---
Target variable encoded: ['Extrovert' 'Introvert'] mapped to [0 1]

--- Starting Final Model Training with Probability Calibration ---
--- Training Fold 1/5 ---
Model wrapped with CalibratedClassifierCV (isotonic).
Fold 1 Accuracy: 0.9698
--- Training Fold 2/5 ---
Model wrapped with CalibratedClassifierCV (isotonic).
Fold 2 Accuracy: 0.9673
--- Training Fold 3/5 ---
Model wrapped with CalibratedClassifierCV (isotonic).
Fold 3 Accuracy: 0.9657
--- Training Fold 4/5 ---
Model wrapped with CalibratedClassifierCV (isotonic).
Fold 4 Accuracy: 0.9709
--- Training Fold 5/5 ---
Model wrapped